<div style="background-color: rgb(15, 15, 35); border: 1px solid rgb(255, 255, 255); color: rgb(204, 204, 204); font-family: monospace; margin-left: -5px; margin-right: -20px; padding: 1rem;"><h2 style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255);">--- Day 7: No Space Left On Device ---</h2><p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">The device the Elves gave you has problems with more than just its communication system. You try to run a system update:</p>
<pre style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); display: inline-block;"><code style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">&#36; system-update --please --pretty-please-with-sugar-on-top
<span title="E099 PROGRAMMER IS OVERLY POLITE" style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Error</span>: No space left on device
</code></pre>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Perhaps you can delete some files to make space for the update?</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:</p>
<pre style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); display: inline-block;"><code style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">&#36; cd /
&#36; ls
dir a
14848514 b.txt
8504156 c.dat
dir d
&#36; cd a
&#36; ls
dir e
29116 f
2557 g
62596 h.lst
&#36; cd e
&#36; ls
584 i
&#36; cd ..
&#36; cd ..
&#36; cd d
&#36; ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
</code></pre>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code>. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Within the terminal output, lines that begin with <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">&#36;</code> are <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">commands you executed</em>, very much like some modern computers:</p>
<ul style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">cd</code> means <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">change directory</em>. This changes which directory is the current directory, but the specific result depends on the argument:
  <ul style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">
  <li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">cd x</code> moves <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">in</em> one level: it looks in the current directory for the directory named <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">x</code> and makes it the current directory.</li>
  <li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">cd ..</code> moves <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">out</em> one level: it finds the directory that contains the current directory, then makes that directory the current directory.</li>
  <li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">cd /</code> switches the current directory to the outermost directory, <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code>.</li>
  </ul>
</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">ls</code> means <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">list</em>. It prints out all of the files and directories immediately contained by the current directory:
  <ul style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">
  <li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">123 abc</code> means that the current directory contains a file named <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">abc</code> with size <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">123</code>.</li>
  <li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);"><code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">dir xyz</code> means that the current directory contains a directory named <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">xyz</code>.</li>
  </ul>
</li>
</ul>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Given the commands and output in the example above, you can determine that the filesystem looks visually like this:</p>
<pre style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); display: inline-block;"><code style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
</code></pre>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Here, there are four directories: <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code> (the outermost directory), <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code> and <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">d</code> (which are in <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code>), and <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">e</code> (which is in <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code>). These directories also contain files of various sizes.</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">total size</em> of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">The total sizes of the directories above can be found as follows:</p>
<ul style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">The total size of directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">e</code> is <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">584</em> because it contains a single file <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">i</code> of size 584 and no other directories.</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">The directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code> has total size <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">94853</em> because it contains files <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">f</code> (size 29116), <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">g</code> (size 2557), and <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">h.lst</code> (size 62596), plus file <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">i</code> indirectly (<code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code> contains <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">e</code> which contains <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">i</code>).</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">d</code> has total size <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">24933642</em>.</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">As the outermost directory, <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code> contains every file. Its total size is <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">48381165</em>, the sum of the size of every file.</li>
</ul>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">To begin, find all of the directories with a total size of <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">at most 100000</em>, then calculate the sum of their total sizes. In the example above, these directories are <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code> and <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">e</code>; the sum of their total sizes is <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);"><em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">95437</em></code> (94853 + 584). (As in this example, this process can count files more than once!)</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Find all of the directories with a total size of at most 100000. <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">What is the sum of the total sizes of those directories?</em></p>
</div>

In [1]:
import numpy as np
import os
import re

In [2]:
with open("./puzzle_inputs/07.txt") as f:
    lines = [line.rstrip() for line in f]

In [3]:
cmd_lines = [i for i, line in enumerate(lines) if line[0] == '$']

In [4]:
file_tree = {}
current_dir = '/'
for i in cmd_lines:
    cmd = lines[i][2:].split(' ')
    if cmd[0] == 'cd':
        current_dir = os.path.abspath(os.path.join(current_dir, cmd[1]))
    elif cmd[0] == 'ls' and current_dir:
        if current_dir in file_tree:
            continue
        file_tree[current_dir] = []
        i += 1
        while i < len(lines) and lines[i][0] != '$':
            size = lines[i].split()[0]
            if size.isnumeric():
                file_tree[current_dir].append(int(size))
            i += 1

In [5]:
directory_sizes = {path: 0 for path in file_tree}
for path in file_tree:
    delta = sum(file_tree[path])
    directory_sizes[path] += delta
    while path != '/':
        path = os.path.abspath(os.path.join(path, '..'))
        directory_sizes[path] += delta

In [6]:
sum(size for size in directory_sizes.values() if size <= 100000)

1490523

<div style="background-color: rgb(15, 15, 35); border: 1px solid rgb(255, 255, 255); color: rgb(204, 204, 204); font-family: monospace; margin-left: -5px; margin-right: -20px; padding: 1rem;"><h2 id="part2" style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255);">--- Part Two ---</h2><p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Now, you're ready to choose a directory to delete.</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">The total disk space available to the filesystem is <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);"><em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">70000000</em></code>. To run the update, you need unused space of at least <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);"><em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">30000000</em></code>. You need to find a directory you can delete that will <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">free up enough space</em> to run the update.</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">In the example above, the total size of the outermost directory (and thus the total amount of used space) is <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">48381165</code>; this means that the size of the <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">unused</em> space must currently be <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">21618835</code>, which isn't quite the <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">30000000</code> required by the update. Therefore, the update still requires a directory with total size of at least <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">8381165</code> to be deleted before it can run.</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">To achieve this, you have the following options:</p>
<ul style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Delete directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">e</code>, which would increase unused space by <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">584</code>.</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Delete directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code>, which would increase unused space by <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">94853</code>.</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Delete directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">d</code>, which would increase unused space by <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">24933642</code>.</li>
<li style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Delete directory <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code>, which would increase unused space by <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">48381165</code>.</li>
</ul>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Directories <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">e</code> and <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">a</code> are both too small; deleting them would not free up enough space. However, directories <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">d</code> and <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">/</code> are both big enough! Between these, choose the <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">smallest</em>: <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);">d</code>, increasing unused space by <code style="border: 1px solid rgb(51, 51, 64); color: rgb(204, 204, 204); background: rgb(16, 16, 26);"><em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">24933642</em></code>.</p>
<p style="border-color: rgb(204, 204, 204); color: rgb(204, 204, 204);">Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. <em style="border-color: rgb(255, 255, 255); color: rgb(255, 255, 255); text-shadow: rgb(255, 255, 255) 0px 0px 5px;">What is the total size of that directory?</em></p>
</div>

In [7]:
TOTAL_DISK_SPACE = 70000000
UNUSED_SPACE = 30000000

In [8]:
min_size_to_delete = directory_sizes['/'] + UNUSED_SPACE - TOTAL_DISK_SPACE
min_size_to_delete

9192532

In [10]:
min(size for size in directory_sizes.values() if size >= min_size_to_delete)

12390492